<img src = 'https://images.manning.com/book/3/8e5d003-09e3-430e-a5a3-f42ee1cafb5f/Stevens-DLPy-HI.png' width = '500' height = '770'>

# 3. It starts with a tensor

The process begins by converting our input into floating-point numbers.

In this chapter, we learn how to deal with all the floating-point numbers in PyTorch by using tensors.

## 3.1 The world as floating-point numbers

We need a way to encode real-world data of the kind we want to process into something digestible by a network

and then decode the output back to something we can understand and use for our purpose.

Intermediate representations are collections of floating-point numbers that characterize the input

and capture the data's structure in a way that is instrumental for describing how inputs are mapped to the outputs of the neural network.

These collections of floating-point numbers and their manipulation are at the heart of modern AI.

It's important to keep in mind that these intermediate representations are the results of

combining the input with the weights of the previous layer of neurons.

Each intermediate representation is unique to the inputs that preceeded it.

Compared to NumPy arrays, PyTorch tensors have a few superpowers, such as the ability to perform very fast operations on

graphical processing units (GPUs), distribute operations on multiple devices or machines,

and keep track of the graph of computations that created them.

We'll start this chapter by introducing PyTorch tensors, covering the basics in order to set things in motion

for our work in the rest of the book.

## 3.2 Tensors: Multidimensional arrays

### 3.2.1 From Python lists to PyTorch tensors

In [1]:
a = [1.0, 2.0, 1.0]

In [2]:
a[0]

1.0

In [3]:
a[2] = 3.0

In [4]:
a

[1.0, 2.0, 3.0]

As we will see in the following chapter, using the more efficient tensor data structure,

many types of data-from images to time series, and even sentences-can be represented.

### 3.2.2 Constructing our first tensors

Let's construct our first PyTorch tensor and see what it looks like.

In [5]:
import torch
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [6]:
a[1]

tensor(1.)

In [7]:
float(a[1])

1.0

In [8]:
a[2] = 2.0
a

tensor([1., 1., 2.])

Althoug on the surface this example doesn't differ much from a list of number objects,

under the hood things are completely different.

### 3.2.3 The essence of tensors

Python lists or tuples of numbers are collections of Python objects that are individually allocated in memory.

PyTorch tensors or NumPy arrays, on the other hand, are views over contiguous memory blocks containing unboxed C numeric types rather than Python objects.

In [9]:
points = torch.zeros(6)

In [10]:
points

tensor([0., 0., 0., 0., 0., 0.])

In [11]:
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

We can also pass a Python list to the constructor, to the same effect.

In [12]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [13]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [14]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [15]:
points.shape

torch.Size([3, 2])

In [16]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [17]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [18]:
points[0, 1]

tensor(1.)

In [19]:
points[0]

tensor([4., 1.])

## 3.3 Indexing tensors

In [20]:
some_list = list(range(6))
some_list[1:4:2]

[1, 3]

In [21]:
points[1 : ]

tensor([[5., 3.],
        [2., 1.]])

In [22]:
points[1:, 0]

tensor([5., 2.])

In [23]:
points[None]

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

## 3.4 Named tensors

The dimensions (or axed) of our tensors usually index something like pixel locations or color channels.

This means when we want to index into a tensor, we need to remember the ordering of the dimensions and write our indexing accordingly.

As data is transformed through multiple tensors, keeping track of which dimension contains what data can be error-prone.

To make things concrete, imagine that we have 3D tensor like img_t from section 2.1.4, and we want to convert it to gray-scale.

In [24]:
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([0.2126, 0.7152, 0.0722])

We also often want our code to generalize - for example, from grayscale images represented as 2D tensors with height and width dimensions to color images adding a third channel dimension (as in RGB),

or from a single image to a batch of images.

In [25]:
batch_t = torch.randn(2, 3, 5, 5)

In [27]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

But now we have the weight, too.

Pytorch will allow us to multiply things that are the same shape, as well as shape where one operand is of size 1 in a given dimension.

It also appends leading dimensions of size 1 automatically.

This is a feature called braodcasting.

batch_t of shape (2, 3, 5, 5) is multiplied by unsqueezed_weights of shape (3, 1, 1), resulting in a tensor of shape (2, 3, 5, 5),

from which we can then sum the third dimnension from the end (the three channels).

In [30]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

Because this gets messy quickly - and for the sake of efficiency - the PyTorch function einsum (adapted from NumPy) specifies an indexing mini-language giving index names to dimensions for sums of such products.

As often in Python, broadcasting - a form of summarizing unnamed things - is done three dots '...';

but don't worry too much about einsum, becuase we will not use it in the following:

In [32]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

PyTorch 1.3 added names tensors as an experimental feature.

In [33]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names = ['channels'])
weights_named

<ipython-input-33-35865d7e5b1b>:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:934.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names = ['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

When we already have a tensor and want to add names, we can call the method refine_names on it.

In [34]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named:', img_named.shape, img_named.names)
print('batch named:', batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [35]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [36]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

If we try to combine dimensions with different names, we get an error

In [37]:
gray_named = (img_named[..., :3] * weights_named).sum('channels')

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.

In [38]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

## 3.5 Tensor element types

Numbers in Python are objects.

Lists in Python are meant for sequential collections fo objects.

The Python interpreter is slow compared to optimized, compiled code.

For these reasons, data science libraries rely on NumPy or introduce dedicated data structures like PyTorch tensors,

which provide efficient low-level implementations of numerical data structures and related operations on them, wrapped in a convenient high-level API.

### 3.5.1 Specifying the numeric type with dtype

### 3.5.2 A dtype for every occasion

### 3.5.3 Managing a tensor's dtype attribute

In [39]:
double_points = torch.ones(10, 2, dtype = torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype = torch.short)

In [40]:
short_points.dtype

torch.int16

In [42]:
double_points.dtype

torch.float64

In [43]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

or the more convenient 'to' method

In [44]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(torch.short)

When mixing input types in operations, the inputs are converted to the larger type automatically

In [45]:
points_64 = torch.rand(5, dtype = torch.double)
points_short = points_64.to(torch.short)
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

## 3.6 The tensor API

In [46]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [47]:
a_t

tensor([[1., 1., 1.],
        [1., 1., 1.]])

Creations ops, Math ops, Indexing slicing joining mutating ops, Random sampling, Serialization, Parallelism...

Take some time to play with the general tensor API.

## 3.7 Tensors: Scenic views of storage

### 3.7.1 Indexing into storage

In [49]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

Even though the tensor reports itself as having three rows and two columns,

the storage under the hood is a contiguous array of size 6.

We can also index into a storage manually. For instance:

In [51]:
points_storage = points.storage()
points_storage[0]

4.0

In [53]:
points.storage()[1]

1.0

In [54]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

### 3.7.2 Modifying stored values: In-place operations

In [55]:
a = torch.ones(3, 2)

In [56]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## 3.8 Tensor metadata: Size, offset, and stride

### 3.8.1 Views of another tensor's storage

In [57]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [58]:
second_point.size()

torch.Size([2])

In [59]:
second_point.shape

torch.Size([2])

In [60]:
points.stride()

(2, 1)

In [61]:
second_point = points[1]
second_point.size()

torch.Size([2])

In [63]:
second_point.storage_offset()

2

In [64]:
second_point.stride()

(1,)

In [65]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [66]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

### 3.8.2 Transposing without copying

In [67]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [68]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [70]:
id(points.storage()) == id(points_t.storage)

# True 나와야 하는데...?

False

In [71]:
points.stride()

(2, 1)

In [72]:
points_t.stride()

(1, 2)

### 3.8.3 Transposing in higher dimensions

In [74]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape

torch.Size([3, 4, 5])

In [75]:
transpose_t.shape

torch.Size([5, 4, 3])

In [76]:
some_t.stride()

(20, 5, 1)

In [77]:
transpose_t.stride()

(1, 5, 20)

### 3.8.4 Contiguous tensors

In [78]:
points.is_contiguous()

True

In [79]:
points_t.is_contiguous()

False

In [81]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [82]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [83]:
points_t.stride()

(1, 2)

In [84]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [85]:
points_t_cont.stride()

(3, 1)

In [86]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]

## 3.9 Moving tensors to the GPU

So far in this chapter, when we've talked about storage, we've meant memory on the CPU.

PyTorch tensors also can be stored on a different kind of processor: a graphics processing unit (GPU).

### 3.9.1 Managing a tensor's device attribute

In [87]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device = 'cuda')

AssertionError: Torch not compiled with CUDA enabled

In [88]:
# torch와 cuda 버전이 안맞는듯 하다...

## 3.10 NumPy interoperability

In [89]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

## 3.11 Generalized tensors are tensors, too

## 3.12 Serializing tensors

## 3.13 Conclusion

## 3.14 Exercises

## 3.15 Summary

- Neural networks transform floating-point representations into other floating-point representations. The starting and ending representations are typically human interpretable, but the intermediate representations are less so.

- These floating-point representations are stored in tensors.

- Tensors are multidimensional arrays; they are the basic data structure in PyTorch.

- PyTorch has a comprehensive standard library for tensor creation, manipulation, and mathematical operations.

- Tensors can be serialized to disk and loaded back.

- All tensor operations in PyTorch can execute on the CPU as well as on the GPU, with no change in the code.

- PyTorch uses a trailing underscore to indicate that a function operates in place on a tensor (for example, Tensor.sqrt_)